In [1]:
# training data and information for this was got at:
# https://towardsdatascience.com/detecting-bad-customer-reviews-with-nlp-d8b36134dc7e

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

testDataPercent = 5
epocs = 10
batchSize = 512
validationSize = 1000

# stems the data
def clean_sen(sen):
    cleaned_train_sentences = []
    stop_words = stopwords.words('english')
    porter = PorterStemmer()

    for x in sen:
        tokens = word_tokenize(x)
        words = [word for word in tokens if word.isalpha()]
        words = [w for w in words if not w in stop_words]
        stemmed = [porter.stem(word) for word in tokens]
        cleaned_train_sentences.append(stemmed)
    return cleaned_train_sentences
        

# turns sentence into list of ints and removes rare words
def text_to_numbers(text, cutoff_for_rare_words = 1):
    
    # Flatten list if sublists are present
    if len(text) > 1:
        flat_text = [item for sublist in text for item in sublist]
    else:
        flat_text = text
    
    # get word freuqncy
    fdist = nltk.FreqDist(flat_text)

    # Convert to Pandas dataframe
    df_fdist = pd.DataFrame.from_dict(fdist, orient='index')
    df_fdist.columns = ['Frequency']

    # Sort by word frequency
    df_fdist.sort_values(by=['Frequency'], ascending=False, inplace=True)

    # Add word index
    number_of_words = df_fdist.shape[0]
    df_fdist['word_index'] = list(np.arange(number_of_words)+1)

    # replace rare words with index zero
    frequency = df_fdist['Frequency'].values
    word_index = df_fdist['word_index'].values
    mask = frequency <= cutoff_for_rare_words
    word_index[mask] = 0
    df_fdist['word_index'] =  word_index
    
    # Convert pandas to dictionary
    word_dict = df_fdist['word_index'].to_dict()
    print(word_dict)
    with open('trainedKeys.json', 'w') as jd:
        json.dump(word_dict, jd)
    
    # Use dictionary to convert words in text to numbers
    text_numbers = []
    for string in text:
        string_numbers = [word_dict[word] for word in string]
        text_numbers.append(string_numbers)  
    
    return (text_numbers)

def pad(data):
    return keras.preprocessing.sequence.pad_sequences(data, value=0, padding='post', maxlen=250)

def trainer(text_numbers_train, train_score):
    # set up neural network
    model = keras.Sequential()
    model.add(keras.layers.Embedding(30000, 16))
    model.add(keras.layers.GlobalAveragePooling1D())
    model.add(keras.layers.Dense(16, activation="relu"))
    model.add(keras.layers.Dense(1, activation="sigmoid"))

    model.summary()

    # set up data loss
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    # split train data into train and validation data
    x_val = text_numbers_train[:validationSize] # strings in numbers
    x_train = text_numbers_train[validationSize:] #  strings in numbers

    y_val = np.asarray(train_score[:validationSize]) # score
    y_train = np.asarray(train_score[validationSize:]) # score

    # train the model
    # fitModel = model.fit(x_train, y_train, batch_size=batchSize, epochs=epocs, validation_data=(x_val, y_val), verbose=1)
    model.fit(x_train, y_train, batch_size=batchSize, epochs=epocs, validation_data=(x_val, y_val), verbose=1)
    # model.fit(x_train, y_train, epochs=epocs)

    model.save("model.h5")
    mod = tf.keras.models.load_model("model.h5")
    mod.summary()
    converter = tf.lite.TFLiteConverter.from_keras_model(mod)
    tflite_model = converter.convert()
    open("converted_model.tflite", "wb").write(tflite_model)
    
    return model

def getdata():
    # read data and store it
    reviews_df = pd.read_csv("Hotel_Reviews.csv")
    testSize = np.ceil(len(reviews_df)/testDataPercent)
    testSize = np.int_(len(reviews_df) - testSize)
    reviews_df["review"] = reviews_df["Negative_Review"] + reviews_df["Positive_Review"]
    reviews = reviews_df["Reviewer_Score"].apply(lambda x: 1 if x < 5 else 0)

    # break data into train and test data
    train_examples, test_examples = reviews_df["review"][0:-testSize], reviews_df["review"][-testSize:-1]
    train_score, test_score = reviews[0:-testSize], reviews[-testSize:-1]
    
    return train_examples, test_examples, train_score, test_score

def nlc():
    train_examples, test_examples, train_score, test_score = getdata()

    cleaned_train_sentences = clean_sen(train_examples)
    cleaned_test_sentences = clean_sen(test_examples)

    # turn all the sentence words to ints
    text_numbers_train = text_to_numbers(cleaned_train_sentences)
    text_numbers_test = text_to_numbers(cleaned_test_sentences)

    # pad the data
    text_numbers_train = pad(text_numbers_train)
    text_numbers_test = pad(text_numbers_test)

    model = trainer(text_numbers_train, train_score)

    # testing the data for accurasy
    results = model.evaluate(text_numbers_test, np.asarray(test_score))

    print(results)

    

nlc()


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\eimhi\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\eimhi\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\eimhi\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\eimhi\miniconda3\envs\tensorflow\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\eimhi\miniconda3\envs\tensorflow\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.

During hand

TypeError: can only concatenate str (not "list") to str